<a href="https://colab.research.google.com/github/dayviddouglas/ciencia_de_dados_licitacoes_prefeitura_do_recife/blob/main/Licita%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa as bibliotecas faz uma requisição do dataset e define o encoding para utf-8 case não consiga executar a requisição retorna um erro

In [1]:
import pandas as pd
import requests as req
from io import StringIO
import urllib
import matplotlib.pyplot as plt

url =  "http://dados.recife.pe.gov.br/dataset/591a6ed4-7beb-4304-a2a1-2af521517a06/resource/c5d7505c-381c-4670-a0c2-1fbf56df50b1/download/dados_abertos_licitacao_concluida.csv"

try:
    response = req.get(url)
    response.encoding = 'utf-8'
    response.raise_for_status()  # Lança um erro para respostas não-sucedidas
    # Usando StringIO para converter o texto em um arquivo em memória e, então, lendo com o Pandas
    csv_raw = StringIO(response.text)
    base = pd.read_csv(csv_raw, sep=';', encoding='utf-8')



except req.RequestException as e:
    print(f"Erro ao acessar o arquivo: {e}")


# Lê o cabeçalho

In [ ]:
base.head(3)

# Informações da base

In [ ]:
base.info()

# Corrigindo tipos das tabelas

In [4]:
base.num_endcontratado.dtypes

dtype('float64')

In [5]:
base.data_ultimafaseprocesso.dtypes

dtype('O')

In [6]:
base.cpf_contratado.dtypes

dtype('float64')

In [7]:
base['data_ultimafaseprocesso'] = pd.to_datetime(base['data_ultimafaseprocesso'])
base['data_aberturaproposta'] = pd.to_datetime(base['data_aberturaproposta'])

In [8]:
base['num_endcontratado'] = base['num_endcontratado'].round().astype('object')

In [9]:
base['cpf_contratado'] = base['cpf_contratado'].astype('object')
base['cnpj_contratado'] = base['cnpj_contratado'].astype('object')

In [10]:
base['valor_total_estimado'] = base['valor_total_estimado'].str.replace(',', '.')
base['valor_licitacao_lote'] = base['valor_licitacao_lote'].str.replace(',', '.')
base['valor_totalhomologadolicitacao'] = base['valor_totalhomologadolicitacao'].str.replace(',', '.')
base['valor_totalhomologadolicitacao'] = base['valor_totalhomologadolicitacao'].fillna(0)
base['valor_total_estimado'] = base['valor_total_estimado'].astype('float')
base['valor_totalhomologadolicitacao'] = base['valor_totalhomologadolicitacao'].astype('float')
base['valor_licitacao_lote'] = base['valor_licitacao_lote'].astype('float')

# Valores nulos

In [ ]:
base.isnull().sum()

In [ ]:
base.isna().sum()

In [ ]:
base.duplicated()

# corrigindo valores nulos nos endereços

In [ ]:
base[base.logradouro_endcontratado.isnull()]

In [ ]:
base[base.bairro_endcontratado.isnull()]

In [ ]:
base[base.uf_endcontratado.isnull()]

In [17]:
base['num_endcontratado'] = base['num_endcontratado'].fillna('S/N')

In [18]:
base['cpf_contratado'] = base['cpf_contratado'].fillna('Empresa')

In [19]:
base['cnpj_contratado'] = base['cnpj_contratado'].fillna('PF')

In [ ]:
base[base.razao_nomecontratado == 'BECKMAN COULTER DO BRASIL LTDA']

Corrigindo empresa BECKMAN COULTER DO BRASIL LTDA

In [21]:
base.loc[base['razao_nomecontratado'] == 'BECKMAN COULTER DO BRASIL LTDA', 'logradouro_endcontratado'] = 'Al. Rio Negro'

In [22]:
base.loc[base['razao_nomecontratado'] == 'BECKMAN COULTER DO BRASIL LTDA', 'num_endcontratado'] = 500

In [23]:
base.loc[base['razao_nomecontratado'] == 'BECKMAN COULTER DO BRASIL LTDA', 'comp_endcontratado'] = '15º andar'

In [24]:
base.loc[base['razao_nomecontratado'] == 'BECKMAN COULTER DO BRASIL LTDA', 'cidade_endcontratado'] = 'Barueri'

In [25]:
base.loc[base['razao_nomecontratado'] == 'BECKMAN COULTER DO BRASIL LTDA', 'uf_endcontratado'] = 'SP'

In [26]:
base.loc[base['razao_nomecontratado'] == 'BECKMAN COULTER DO BRASIL LTDA', 'bairro_endcontratado'] = 'Alphaville Industrial'

Corrigindo empresa FICAPOÇOS FIOS E CABOS LTDA.

http://cnpj.info/10376365000179

In [ ]:
base[base.razao_nomecontratado == 'FICAPOÇOS FIOS E CABOS LTDA.']

In [28]:
base.loc[base['razao_nomecontratado'] == 'FICAPOÇOS FIOS E CABOS LTDA.', 'logradouro_endcontratado'] = 'RUA UM'

In [29]:
base.loc[base['razao_nomecontratado'] == 'FICAPOÇOS FIOS E CABOS LTDA.', 'bairro_endcontratado'] = 'DISTRITO INDUSTRIAL'

In [30]:
base.loc[base['razao_nomecontratado'] == 'FICAPOÇOS FIOS E CABOS LTDA.', 'cidade_endcontratado'] = 'POCOS DE CALDAS'

In [31]:
base.loc[base['razao_nomecontratado'] == 'FICAPOÇOS FIOS E CABOS LTDA.', 'uf_endcontratado'] = 'MG'

Corrigindo empresa STEMAC SA GRUPOS GERADORES

http://cnpj.info/Stemac-Sa-Grupos-Geradores-Em-Recuperacao-Judicial-Anchieta-Porto-Alegre


In [ ]:
base[base.razao_nomecontratado == 'STEMAC SA GRUPOS GERADORES']

In [33]:
base.loc[base['razao_nomecontratado'] == 'STEMAC SA GRUPOS GERADORES', 'logradouro_endcontratado'] = 'AVENIDA SEVERO DULLIUS'

In [34]:
base.loc[base['razao_nomecontratado'] == 'STEMAC SA GRUPOS GERADORES', 'bairro_endcontratado'] = 'ANCHIETA'

In [35]:
base.loc[base['razao_nomecontratado'] == 'STEMAC SA GRUPOS GERADORES', 'cidade_endcontratado'] = 'PORTO ALEGRE'

In [36]:
base.loc[base['razao_nomecontratado'] == 'STEMAC SA GRUPOS GERADORES', 'uf_endcontratado'] = 'RS'

In [37]:
base.loc[base['razao_nomecontratado'] == 'STEMAC SA GRUPOS GERADORES', 'comp_endcontratado'] = 'Sala 1101-2'

In [38]:
base.loc[base['razao_nomecontratado'] == 'STEMAC SA GRUPOS GERADORES', 'num_endcontratado'] = 1395

Corrigindo empresa FML TECNOLOGIA ELETRONICA - EIRELI

http://cnpj.info/Fml-Tecnologia-Eletronica-Eireli-Fml-Tecnologia

In [ ]:
base[base.razao_nomecontratado == 'FML TECNOLOGIA ELETRONICA - EIRELI']

In [40]:
base.loc[base['razao_nomecontratado'] == 'FML TECNOLOGIA ELETRONICA - EIRELI', 'bairro_endcontratado'] = 'IBURA'

Corrigindo endereços por cidade

In [41]:
base.loc[base['logradouro_endcontratado'] == 'RUA DA OLARIA', 'bairro_endcontratado'] = 'TORRÕES'

In [42]:
base.loc[base['cidade_endcontratado'] == 'BELO HORIZONTE', 'uf_endcontratado'] = 'MG'

In [43]:
base.loc[base['logradouro_endcontratado'] == 'RUA LINDOLFO DE AZEVEDO', 'bairro_endcontratado'] = 'JARDIM AMÉRICA'

In [44]:
base.loc[base['logradouro_endcontratado'] == 'RUA CLÉLIA MANFRO, 1843', 'logradouro_endcontratado'] = 'RUA CLÉLIA MANFRO'

In [45]:
base.loc[base['logradouro_endcontratado'] == 'RUA CLÉLIA MANFRO', 'num_endcontratado'] = '1843'

In [46]:
base.loc[base['logradouro_endcontratado'] == 'RUA BENJAMIN CONSTANT', 'bairro_endcontratado'] = 'LARGO DO BARRADAS'

In [47]:
base.loc[base['logradouro_endcontratado'] == 'RUA RUTH FRANCO ROCHA', 'bairro_endcontratado'] = 'SÃO LOURENÇO'

In [48]:
base.loc[base['logradouro_endcontratado'] == 'RUA CEL MAURÍCIO GONÇALVES MOREIRA', 'bairro_endcontratado'] = 'CENTRO'

In [49]:
base.loc[base['cidade_endcontratado'] == 'CAETÉ', 'uf_endcontratado'] = 'MG'

In [50]:
base.loc[base['cidade_endcontratado'] == 'GOIANA', 'uf_endcontratado'] = 'PE'

In [51]:
base.loc[base['cidade_endcontratado'] == 'JABOATÃO DOS GUARARAPES', 'uf_endcontratado'] = 'PE'

In [52]:
base.loc[base['cidade_endcontratado'] == 'SÃO BERNARDO', 'uf_endcontratado'] = 'SP'

In [53]:
base.loc[base['cidade_endcontratado'] == 'CAXIAS DO SUL', 'uf_endcontratado'] = 'RS'

Corrigindo empresa ARINOS COMÉRCIO DE MATERIAIS DE CONSTRUÇÃO LTDA

http://cnpj.info/29616156000183

In [54]:
base.loc[base['razao_nomecontratado'] == 'ARINOS COMÉRCIO DE MATERIAIS DE CONSTRUÇÃO LTDA', 'bairro_endcontratado'] = 'JARDIM FLORIDA'


Corrigindo empresa FOCO PRODUCOES LTDA

http://cnpj.info/Foco-Producoes-Foco-3-Producoes

In [55]:
base.loc[base['razao_nomecontratado'] == 'FOCO PRODUCOES LTDA', 'bairro_endcontratado'] = 'PETROPOLIS'

Corrigindo empresa JRIBEIRO COMERCIO ATACADISTA LDTA

http://cnpj.info/J-Ribeiro-Comercio-Atacadista-Ltda

In [56]:
base.loc[base['razao_nomecontratado'] == 'JRIBEIRO COMERCIO ATACADISTA LDTA', 'bairro_endcontratado'] = 'JARDIM BELA SUICA'

<h3> Filtragem de dados</h3>

<h5>Quantidade de processos licitatórios de 2023 até a atualidade.</h5>

In [57]:
base[base["ano_processolicitatorio"]>= 2023].shape[0]




2333

In [ ]:
qtd_process_por_uf=base["uf_endcontratado"].value_counts()
qtd_process_por_uf

# Quantidade de processo por uf do maior para o menor.




In [ ]:
qtd_process_por_uf.plot(kind='barh', figsize=(10, 10))
plt.title('Quantidade de processos por estado.')
plt.xlabel('Processos')
plt.ylabel('Cidades')
plt.gca().invert_yaxis()
plt.xticks(rotation=90, fontsize=8)
plt.show()

In [ ]:
process_pe=base[base["uf_endcontratado"]== "PE"]
process_pe["cidade_endcontratado"].value_counts()
# Quantidade de processos em pernambuco por cidades.


In [ ]:
process_pe["cidade_endcontratado"].value_counts().plot(kind='barh', figsize=(10, 10))
plt.title('Quantidade de processos em pernambuco por cidades.')
plt.xlabel('Processos')
plt.ylabel('Cidades')
plt.gca().invert_yaxis()
plt.xticks(rotation=90, fontsize=8)
plt.show()

In [ ]:
base[base["modalidadeprocessolicitatorio"]!= "PREGÃO ELETRÔNICO"]
# Os processos que não são da modalidade pregão eletrônico.

In [ ]:
base["comissao_licitacao"].value_counts()

# Quantidade de processos em relação a sua respectiva comissão.


In [ ]:
base["comissao_licitacao"].value_counts().plot(kind='barh', figsize=(10, 10))
plt.title('Frequência de comissões por licitação')
plt.xlabel('Frequência')
plt.ylabel('Comissões')
plt.gca().invert_yaxis()
plt.xticks(rotation=90, fontsize=8)
plt.show()

In [ ]:
base["orgao_licitante"].value_counts()
# quantidade de licitações por orgãos licitantes.



In [ ]:
base["orgao_licitante"].value_counts().plot(kind='barh', figsize=(10, 10))
plt.title('Licitações por orgãos')
plt.xlabel('Frequência')
plt.ylabel('Licitações')
plt.gca().invert_yaxis()
plt.xticks(rotation=90, fontsize=8)
plt.show()

In [ ]:
base["objeto"].value_counts()
# Quantidade de licitações para aquisição de materiais.

In [ ]:
qtd_process_maior_2022=base[base["ano_processolicitatorio"]>= 2022]
qtd_process_maior_2022["objeto"].value_counts()

# quantidade de objetos mais solicitados de apartir do ano de 2022 até a atualidade.

<h3> Estrutura do Dataset</h3>

In [69]:
base.shape
# Quantidade de linhas e colunas do dataset.

(19131, 23)

<h3> Análise Estatística</h3>

<h5> Valor Total Homologado. </h5>

In [ ]:
# prompt: Crie um gráfico com os valores do valor total homologado em relação a sua respectiva frequência. Faça uma restrição dos 5 mais frequentes...

base["valor_totalhomologadolicitacao"].value_counts().head(5).plot(kind='barh', figsize=(10, 10))
plt.title('Valor Total Homologado')
plt.xlabel('Frequência')
plt.ylabel('Valor')
plt.gca().invert_yaxis()
plt.xticks(rotation=90, fontsize=8)
plt.show()


<h6>  Moda  </h6>

In [71]:
base["valor_totalhomologadolicitacao"].mode()

0    48981781.66
Name: valor_totalhomologadolicitacao, dtype: float64

<h6>  Média  </h6>

In [72]:
base["valor_totalhomologadolicitacao"].mean()

4949676.153772411

<h6>  Médiana </h6>

In [73]:
base["valor_totalhomologadolicitacao"].median()

934045.55

<h6> Variância </h6>

In [74]:
base["valor_totalhomologadolicitacao"].var()

336869848544124.4

<h6>Desvio Padrão</h6>

In [75]:
base["valor_totalhomologadolicitacao"].std()

18354014.507570937

<h6> Assimetria: Sendo ela positiva, indicando uma cauda mais longa ou mais espessa à direita. </h6>

In [76]:
base["valor_totalhomologadolicitacao"].skew()

33.641281184396206

<h6> Curtose: <strong>Leptocúrtica</strong>,pois tem um excesso de curtose positiva.</h6>

In [77]:
base["valor_totalhomologadolicitacao"].kurtosis()

1694.6579564678113

<h5> Valor de licitação por lote </h5>

In [ ]:
base["valor_licitacao_lote"].value_counts().head(5).plot(kind='barh', figsize=(10, 10))
plt.title('Valor de licitação por lote')
plt.xlabel('Frequência')
plt.ylabel('Valor')
plt.gca().invert_yaxis()
plt.xticks(rotation=90, fontsize=8)
plt.show()

<h6> Moda</h6>

In [ ]:
base["valor_licitacao_lote"].mode()

<h6>  Média  </h6>

In [ ]:
base["valor_licitacao_lote"].mean()

<h6>  Mediana  </h6>

In [ ]:
base["valor_licitacao_lote"].median()

<h6> Variância </h6>

In [82]:
base["valor_licitacao_lote"].var()

89619700094276.73

<h6>Desvio Padrão</h6>

In [83]:
base["valor_licitacao_lote"].std()

9466768.196923211

<h6> Assimetria: Sendo ela positiva, indicando uma cauda mais longa ou mais espessa à direita. </h6>

In [ ]:
base["valor_licitacao_lote"].skew()

<h6> Curtose: <strong>Leptocúrtica</strong>,pois tem um excesso de curtose positiva.</h6>

In [ ]:
base["valor_licitacao_lote"].kurtosis()